## MDE Application to Fly Neural Data

### Import modules
Required modules are pyEDM, pandas & pyplot.

In [ ]:
from pyEDM      import Simplex, ComputeError
from pandas     import read_csv, DataFrame
from matplotlib import pyplot as plt

%matplotlib ipympl

These modules need to be available for the`Evaluate()`function to compare MDE results. 

```python
import numpy
import sklearn
import pydiffmap
```

#### Import MDE & Evaluate classes

In [ ]:
from dimx import MDE, Evaluate

---

### Drosophila melanogaster

A fly expressing the calcium indicator GCaMP6f as a measure of neuronal activity was
recorded walking on a Styrofoam ball. Neuronal activity was segmented by independent
component analysis (ICA) yielding 80 component brain areas. Two behavioral
variables were simultaneously recorded: forward speed (FWD) and left/right turning
speed (Left Right)

In [ ]:
df = read_csv( '../data/Fly80XY_norm_1061.csv' )

#### Instantiate and Run MDE class objects for FWD & Left_Right targets
We use the first 300 time series rows to create the EDM library, and perform out-of-sample prediction on time series rows 301-600. These indices are Not zero offset. 

`ccmSlope` is the minimum slope of a linear fit to the CCM rho(L) curve to validate a causal driver. L is the vector of CCM library sizes at which CCM is evaluated. Default values for L are percentiles [10,15,85,90] of the number of observations (rows).

In [ ]:
Fly_FWD = MDE( df,                     # Pandas DataFrame of observables
               target = 'FWD',         # target behavior variable
               removeColumns = ['index','FWD','Left_Right'], # variables to ignore
               D    = 12,              # Max number of dimensions
               lib  = [1,300],         # EDM library start,stop indices
               pred = [301,600],       # EDM prediction start,stop indices
               ccmSlope = 0.01,        # CCM convergence criteria
               embedDimRhoMin = 0.65,  # Minimum rho for CCM embedding dimension
               crossMapRhoMin = 0.5,   # Minumum rho for cross map of target : variables
               cores = 10,             # Number of cores in CrossMapColumns()
               chunksize = 30,
               plot = False )
Fly_FWD.Run()

In [ ]:
Fly_LR = MDE( df,
              target = 'Left_Right',
              removeColumns = ['index','FWD','Left_Right'], 
              D    = 12,
              lib  = [1,600],
              pred = [801,1000], 
              ccmSlope = 0.01,
              embedDimRhoMin = 0.2,
              crossMapRhoMin = 0.05,
              cores = 10,
              chunksize = 30,
              plot = False )
Fly_LR.Run()

#### Combine FWD & Left_Right results to DataFrame and plot

In [ ]:
FlyDF = DataFrame( 
    {'D' : [i for i in range(1, len(Fly_FWD.MDEOut)+1)],
     'FWD_variable':Fly_FWD.MDEOut['variables'], 'FWD_rho':Fly_FWD.MDEOut['rho'], 
     'LR_variable' :Fly_LR.MDEOut['variables'],  'LR_rho' :Fly_LR.MDEOut['rho'] } )

In [ ]:
ax = FlyDF.plot( x = 'D', y=['FWD_rho','LR_rho'], label = ['FWD','Left_Right'],
                 lw = 3, figsize = (5,4), title = 'Fly MDE' )
table = ax.table( cellText = FlyDF.loc[:,['D','FWD_variable','LR_variable']].values,
                  colLabels = ['D','FWD','LR'], loc = 'lower center', fontsize = 10,
                  colWidths = [0.1,0.12,0.12], edges = 'open', colLoc = 'left', cellLoc = 'left' )

#### Evaluate MDE components & compare to PCA & Diffusion Map

In [ ]:
Fly_FWD_Eval = Evaluate( df, 
                         columns_range = [1,81], # 0-offset range of columns for PCA, DMap
                         mde_columns = ['TS33', 'TS4', 'TS8', 'TS9', 'TS32'],
                         predictVar = 'FWD',
                         library    = [1, 300],   # index start,stop of observations for library
                         prediction = [301, 600], # index start,stop of predictions 
                         components = 5,          # Number of PCA & DMap components
                         dmap_k     = 15,         # diffusion_map k nearest neighbors
                         figsize    = (8,6) )
Fly_FWD_Eval.Run()
Fly_FWD_Eval.Plot()

---

#### MDE Arguments

| Parameter | Default | Description |
|---|---|---|
| dataFrame | None | Pandas DataFrame : column observation vectors, row observations |
| dataFile | None | Data file name to load |
| dataName | None | Data name in .npz archive |
| removeTime | False | Remove first column from dataFrame |
| noTime | False | First column of dataFrame is not time vector |
| columnNames | [] | dataFrame columns to process |
| initDataColumns | [] | If reading .npz omit these leading columns |
| removeColumns | [] | Columns to ignore |
| D | 3 | Maximum number of MDE dimensions |
| target | None | Target variable |
| lib | [] | EDM library indices. Default to all rows |
| pred | [] | EDM prediction indices. Default to all rows  |
| Tp | 1 | Prediction time interval |
| tau | -1 | CCM embedding time delay |
| exclusionRadius | 0 | CCM library temporal exlcusion radius |
| sample | 20 | CCM random library samples to average |
| pLibSizes | [10, 15, 85, 90] | Percentiles of CCM library sizes |
| noCCM | False | Disable CCM |
| ccmSlope | 0.01 | Slope of CCM(LibSizes) convergence |
| ccmSeed | None | CCM random seed |
| E | 0 | CCM embedding dimension. If 0 compute automatically |
| crossMapRhoMin | 0.5 | Minimum rho for cross map acceptance |
| embedDimRhoMin | 0.5 | Minimum rho for CCM embedding dimension |
| maxE | 15 | Maximum embedding dimension for CCM |
| firstEMax | False | CCM embedding dimension is first local peak in rho(E) |
| timeDelay | 0 | Add N=timeDelay time delays |
| cores | 5 | number of multiprocessing CPU in CrossMapColumns() |
| outDir | None | Output file directory |
| outFile | None | MDE object pickle file |
| outCSV | None | CSV of MDE output |
| logFile | None | Log file |
| consoleOut | True | Echo output to console |
| verbose | False | Verbose mode |
| debug | False | Debug mode |
| plot | False | Plot MDE result |
| title | None | Plot title |
| args | None | ArgumentParser object from CLI_Parser |